Let's import all the libraries needed for the project

In [1]:
!pip install bs4

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.cluster import KMeans
import requests

Let's read the page of Toronto's neighborhoods

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

Let's parse the url with the soup object
Assumption: Based on the page - the main table appears in the first table of the page.

In [4]:
tables = soup.findAll("table")
torontos_negihboarhoods_table = tables[0]
col_names = [header.get_text()[:-1] for header in torontos_negihboarhoods_table.findAll("th")] #take all the content without the '\n'
table_content = torontos_negihboarhoods_table.findAll("tr")[1:] 
table_data = list()
for line in table_content:
    line_content = line.findAll("td")
    table_data = table_data + [[cell.get_text()[:-1] for cell in line_content]] #take all the content without the '\n'
    
neighborhoods_df = pd.DataFrame(columns = col_names, data= table_data)
neighborhoods_df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Let's clean the table according to the intructions give in the assignment

In [5]:
neighborhoods_df = neighborhoods_df[neighborhoods_df['Borough']!='Not assigned']
neighborhoods_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Let's show the shape of the df

In [6]:
neighborhoods_df.shape
neighborhoods_df = neighborhoods_df.reset_index(drop=True)
neighborhoods_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
loc_df = pd.read_csv('Geospatial_Coordinates.csv')

In [11]:
neighborhoods_df = loc_df.join(neighborhoods_df, lsuffix='_caller', rsuffix='_other').reset_index(drop=True)

In [13]:
neighborhoods_df = neighborhoods_df.drop(['Postal Code_other'],axis=1)
neighborhoods_df = neighborhoods_df.rename(columns={"Postal Code_caller": "Postal Code"})
neighborhoods_df

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,North York,Parkwoods
1,M1C,43.784535,-79.160497,North York,Victoria Village
2,M1E,43.763573,-79.188711,Downtown Toronto,"Regent Park, Harbourfront"
3,M1G,43.770992,-79.216917,North York,"Lawrence Manor, Lawrence Heights"
4,M1H,43.773136,-79.239476,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M9P,43.696319,-79.532242,Downtown Toronto,Church and Wellesley
100,M9R,43.688905,-79.554724,East Toronto,"Business reply mail Processing Centre, South C..."
101,M9V,43.739416,-79.588437,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
